In [4]:
import requests
import urllib
import ftplib
from datetime import datetime, date

In [12]:
# Reference docs for the format and organization of ADP VIIRS data 
# https://www.star.nesdis.noaa.gov/smcd/spb/aq/AerosolWatch/docs/EPS_ADP_Users_Guide_V1_Sep2020.pdf

# Bounding box values for CA
# https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/
# original bounding box (lat, lon)
UPPER_RIGHT = (43, -114)
LOWER_LEFT = (32, -125)

# Need to extend the bounding box region, since the swath width of VIIRS granule is 3060 km.
LAT_OFFSET = 10 
LON_OFFSET = 20

MAXLAT = UPPER_RIGHT[0] + LAT_OFFSET
MAXLON = UPPER_RIGHT[1] + LON_OFFSET
MINLAT = LOWER_LEFT[0] - LAT_OFFSET
MINLON = LOWER_LEFT[1] - LON_OFFSET

# API URL for OrbNav API
ORBNAV_API_URL = "https://sips.ssec.wisc.edu/orbnav/api/v1/boxtimes.json?"

# Satellite IDs used in the OrbNav API
SATELLITE_ID_MAP = {
    "NOAA_20": 43013,
    "SNPP": 37849
}

# FTP hostname 
FTP_HOSTNAME = "ftp.star.nesdis.noaa.gov"

# Filepath on the FTP server for the two VIIRS satellites
SATELLITE_FTP_PATH_MAP = {
    "NOAA_20": "/pub/smcd/hzhang/VIIRS_EPS_NRT/ADP_N20/CONUS/",
    "SNPP": "/pub/smcd/hzhang/VIIRS_EPS_NRT/ADP/CONUS/"
}

In [13]:
def fetch_overpass_times(satellite: str, date: datetime = datetime.utcnow()):
    # Returns a list of (start, end) datetime objects for a satellite using the OrbNav API
    satellite_id = SATELLITE_ID_MAP[satellite]
    ur = str(MAXLAT) + ", " + str(MAXLON)
    ll = str(MINLAT) + ", " + str(MINLON)
    args = {
        "sat": satellite_id,
        "start": get_utc_start(date),
        "end": get_utc_end(date),
        "ll": ll,
        "ur": ur
    }
    request_url = ORBNAV_API_URL + urllib.parse.urlencode(args)
    resp = requests.get(request_url)
    if resp.status_code != 200:
        return
    return process_resp(resp.json())

def get_utc_start(curr_date: datetime):
    # return midnight timestamp of  date
    midnight = curr_date.replace(hour=0, minute=0, second=0, microsecond=0)
    return midnight.strftime("%Y-%m-%dT%H:%M:%SZ")

def get_utc_end(curr_date: datetime):
    # return latest timestamp of  date
    midnight = curr_date.replace(hour=23, minute=59, second=59, microsecond=0)
    return midnight.strftime("%Y-%m-%dT%H:%M:%SZ")

def process_resp(overpass_times_json):
    # processes the request and creates a list of (start, end) datetime objects that correspond to when the satellite
    # was in the bounding box
    overpass_times = []
    for data in overpass_times_json["data"]:
        start_timestamp = data[0][0]
        end_timestamp = data[1][0]
        start = datetime.strptime(start_timestamp, '%Y-%m-%dT%H:%M:%SZ')
        end = datetime.strptime(end_timestamp, '%Y-%m-%dT%H:%M:%SZ')
        overpass_times.append([start, end])
    return overpass_times

def is_valid_granule(filename, overpass_times):
    observation_timestamp = get_observation_timestamp(filename)
    for overpass in overpass_times:
        start, end = overpass
        if start <= observation_timestamp <= end: 
            return True
    return False

def get_observation_timestamp(filename):
    # Returns a datetime object corresponding to the file observation time
    observation_timestamp = filename.split("_")[3][1:]
    observation_datetime = datetime.strptime(observation_timestamp, '%Y%m%d%H%M%S%f').replace(microsecond=0)
    return observation_datetime


def download_granules(satellite, download_dir = "", date: datetime = datetime.utcnow()):
    # Downloads the granules from the FTP server for a given satellite for timestamps covering California
    ftp_path = SATELLITE_FTP_PATH_MAP[satellite]
    ftp_path += date.strftime("%Y%m%d")
    
    overpass_times = fetch_overpass_times(satellite, date)
    
    with ftplib.FTP(FTP_HOSTNAME) as ftp:
        try:
            ftp.login()
            ftp.cwd(ftp_path)
            files = []
            ftp.dir(files.append)
            for file in files:
                filename = file.split(" ")[-1]
                if (is_valid_granule(filename, overpass_times)):
                    with open(download_dir + filename, 'wb') as f:
                        print("Downloading %s..." % filename)
                        ftp.retrbinary('RETR ' + filename, f.write)
        except ftplib.all_errors as e:
            print('FTP error:', e)

In [14]:
nov12 = datetime(2020, 11, 12)

In [15]:
fetch_overpass_times("NOAA_20", nov12)

[[datetime.datetime(2020, 11, 12, 7, 58, 18),
  datetime.datetime(2020, 11, 12, 7, 59, 41)],
 [datetime.datetime(2020, 11, 12, 9, 32, 18),
  datetime.datetime(2020, 11, 12, 9, 41, 11)],
 [datetime.datetime(2020, 11, 12, 11, 13, 48),
  datetime.datetime(2020, 11, 12, 11, 21, 34)],
 [datetime.datetime(2020, 11, 12, 19, 15, 45),
  datetime.datetime(2020, 11, 12, 19, 20, 52)],
 [datetime.datetime(2020, 11, 12, 20, 53, 28),
  datetime.datetime(2020, 11, 12, 21, 2, 22)],
 [datetime.datetime(2020, 11, 12, 22, 34, 58),
  datetime.datetime(2020, 11, 12, 22, 38, 59)]]

In [ ]:
download_granules("NOAA_20", "",  nov12)

In [ ]:
fetch_overpass_times("SNPP", nov12)

In [ ]:
download_granules("SNPP", "",  nov12)